In [19]:
import numpy as np
import pandas as pd
import re
import pandas_gbq as bq
import glob
import errno
import warnings
import os
import collections
import smart_open
import random

warnings.filterwarnings("ignore")

In [82]:
df = pd.read_csv('/Users/tommybai/Downloads/histories_08012018.csv')
del df['Unnamed: 0']

In [16]:
click_hist = df[['CampaignID','URL', 'clicked','EmailAddress']].drop_duplicates()

In [49]:
click_hist['URL_cut'] = click_hist['URL'].apply(lambda x: x.split('buzzfeed.com/')[-1])
click_hist['URL_cut'] = click_hist['URL_cut'].apply(lambda x: x.split('?origin=tastynl')[0])

In [89]:
click_hist.to_csv('click_hist_full.csv')

In [96]:
### query from big query
df = bq.read_gbq(
'''
SELECT * FROM `bf-data-sci.z_tommy.hw18_click_hist_cleaned`
WHERE buzz_id IS NOT NULL
'''
, project_id = 'bf-data-sci', dialect = 'standard')

In [32]:
df = pd.DataFrame()
path = '/Users/tommybai/Desktop/hw18-newsletter-rec/hw18_cleaned_data/data/tommy*'
files = glob.glob(path)
for name in files:
    try:
        with open(name) as f:
            data_df = pd.read_json(f, lines=True)
            print('opened one file')
            df = df.append(data_df, ignore_index=True)
            df.shape
    except IOError as exc:
        if exc.errno != errno.EISDIR:
            raise

opened one file
opened one file
opened one file
opened one file
opened one file
opened one file
opened one file
opened one file
opened one file
opened one file


In [33]:
df.shape

(6913642, 14)

In [84]:
from gensim.models import Word2Vec, FastText, TfidfModel, Phrases, KeyedVectors
from gensim.utils import simple_preprocess, tokenize
from gensim import corpora
from gensim.models.doc2vec import LabeledSentence
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from scipy.spatial.distance import cosine
from sklearn.manifold import TSNE

In [47]:
df.head()

,badges,bf_url,blurb,buzz_id,campaignID,clicked,cm_url,description,emailaddress,post_title,primary_buzz_vertical,published_at,tags,url_cut
0,"[lol, viral]",mjkiebus/it-was-an-honest-mistake,Whoops.,4867913,97fe299bb427b0bc12523305ac47750a,0,https://www.buzzfeed.com/mjkiebus/it-was-an-ho...,Original reporting and analysis plus the real ...,liz.fitz@icloud.com,21 Things That'll Make You Sigh And Laugh,BuzzFeed,2018-07-19 15:16:03,"[fails, face palm, mistakes, facebook fails, f...",mjkiebus/it-was-an-honest-mistake
1,"[lol, viral]",mjkiebus/it-was-an-honest-mistake,Whoops.,4867913,97fe299bb427b0bc12523305ac47750a,0,https://www.buzzfeed.com/mjkiebus/it-was-an-ho...,Original reporting and analysis plus the real ...,shira9111@gmail.com,21 Things That'll Make You Sigh And Laugh,BuzzFeed,2018-07-19 15:16:03,"[fails, face palm, mistakes, facebook fails, f...",mjkiebus/it-was-an-honest-mistake
2,"[lol, viral]",mjkiebus/it-was-an-honest-mistake,Whoops.,4867913,97fe299bb427b0bc12523305ac47750a,0,https://www.buzzfeed.com/mjkiebus/it-was-an-ho...,Original reporting and analysis plus the real ...,huebbe@sbcglobal.net,21 Things That'll Make You Sigh And Laugh,BuzzFeed,2018-07-19 15:16:03,"[fails, face palm, mistakes, facebook fails, f...",mjkiebus/it-was-an-honest-mistake
3,"[lol, viral]",mjkiebus/it-was-an-honest-mistake,Whoops.,4867913,97fe299bb427b0bc12523305ac47750a,0,https://www.buzzfeed.com/mjkiebus/it-was-an-ho...,Original reporting and analysis plus the real ...,jhoannachan@yahoo.com,21 Things That'll Make You Sigh And Laugh,BuzzFeed,2018-07-19 15:16:03,"[fails, face palm, mistakes, facebook fails, f...",mjkiebus/it-was-an-honest-mistake
4,"[lol, viral]",mjkiebus/it-was-an-honest-mistake,Whoops.,4867913,97fe299bb427b0bc12523305ac47750a,0,https://www.buzzfeed.com/mjkiebus/it-was-an-ho...,Original reporting and analysis plus the real ...,ilieva.emilija@gmail.com,21 Things That'll Make You Sigh And Laugh,BuzzFeed,2018-07-19 15:16:03,"[fails, face palm, mistakes, facebook fails, f...",mjkiebus/it-was-an-honest-mistake


In [98]:
df.shape

(6913642, 14)

In [143]:
def read_corpus(data, tokens_only=False):
    for i in range(data.shape[0]):
        line = data['post_title'][i] + data['description'][i]
        if tokens_only:
            yield gensim.utils.simple_preprocess(line)
        else:
            # For training data, add tags
            tag = data['buzz_id'][i]
            yield gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(line), [tag])

In [99]:
gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(df['post_title'][0] + df['description'][0]),[df['buzz_id'][0]])

TaggedDocument(words=['things', 'that', 'll', 'make', 'you', 'sigh', 'and', 'laughoriginal', 'reporting', 'and', 'analysis', 'plus', 'the', 'real', 'conversation', 'of', 'the', 'social', 'web'], tags=[4867913])

In [144]:
train_corpus = list(read_corpus(df))
#test_corpus = list(read_corpus(df, tokens_only=True))

In [181]:
train_corpus[0]

TaggedDocument(words=['things', 'that', 'll', 'make', 'you', 'sigh', 'and', 'laughoriginal', 'reporting', 'and', 'analysis', 'plus', 'the', 'real', 'conversation', 'of', 'the', 'social', 'web'], tags=[4867913])

In [146]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=2, epochs=40)

In [147]:
model.build_vocab(train_corpus)

In [150]:
model.wv.vocab

{'things': <gensim.models.keyedvectors.Vocab at 0x202d6a4a90>,
 'that': <gensim.models.keyedvectors.Vocab at 0x202d6a4cc0>,
 'll': <gensim.models.keyedvectors.Vocab at 0x202d6a49e8>,
 'make': <gensim.models.keyedvectors.Vocab at 0x202d6a4908>,
 'you': <gensim.models.keyedvectors.Vocab at 0x202d6a4a58>,
 'sigh': <gensim.models.keyedvectors.Vocab at 0x202d6a4898>,
 'and': <gensim.models.keyedvectors.Vocab at 0x202d6a4ac8>,
 'laughoriginal': <gensim.models.keyedvectors.Vocab at 0x202d6a4a20>,
 'reporting': <gensim.models.keyedvectors.Vocab at 0x202d6a4940>,
 'analysis': <gensim.models.keyedvectors.Vocab at 0x202d6a4c50>,
 'plus': <gensim.models.keyedvectors.Vocab at 0x202d6a4dd8>,
 'the': <gensim.models.keyedvectors.Vocab at 0x202d6a4cf8>,
 'real': <gensim.models.keyedvectors.Vocab at 0x202d6a4c88>,
 'conversation': <gensim.models.keyedvectors.Vocab at 0x202d6a4eb8>,
 'of': <gensim.models.keyedvectors.Vocab at 0x202d6a4ef0>,
 'social': <gensim.models.keyedvectors.Vocab at 0x202d6a4f28>,
 

In [156]:
model.infer_vector(['Things', 'That', 'Make', 'You', 'Sigh', 'And', 'Laugh'])

array([ 0.00371606, -0.00345182, -0.00115388,  0.00886875, -0.00472428,
       -0.0086563 , -0.00706672, -0.00014137,  0.00598039,  0.00692708,
        0.00919141,  0.00083214,  0.00355998, -0.00191095,  0.00568263,
        0.00538977,  0.00680876, -0.00343867, -0.00735471, -0.00765289,
       -0.00624519, -0.00561222,  0.00306099, -0.00499362, -0.00932099,
        0.00972205, -0.00813914, -0.00185651,  0.00487733, -0.00560244,
        0.00976653, -0.00706185, -0.00453003, -0.00091302, -0.00092648,
        0.00277097,  0.00994452, -0.00737987,  0.00507416, -0.00638279,
        0.00375948, -0.00864376,  0.00767996,  0.00217919, -0.00273862,
       -0.00668225,  0.00806411, -0.00762317,  0.00069004,  0.00604061],
      dtype=float32)

In [165]:
# to remove duplicate elements
def Remove(duplicate):
    final_list = []
    for num in duplicate:
        if num not in final_list:
            final_list.append(num)
    return final_list
     
train_corpus_unq = Remove(train_corpus)

In [167]:
%time model.train(train_corpus_unq, total_examples=model.corpus_count, epochs=40)

CPU times: user 132 ms, sys: 25.1 ms, total: 157 ms
Wall time: 154 ms


In [178]:
np.savetxt('train_corpus_unq.txt', train_corpus_unq, fmt='%s')

In [171]:
model.infer_vector(['11','Things', 'That', 'Make', 'You', 'Sigh', 'And', 'Laugh'])

array([-0.00790361,  0.00097407,  0.0062989 , -0.00215488,  0.00580458,
        0.00260498,  0.00330859,  0.00714086,  0.00218126,  0.00119387,
       -0.00083054,  0.00933671, -0.00827288,  0.00740052,  0.00068578,
       -0.00262523, -0.00132894, -0.00477994,  0.00429952,  0.00678763,
       -0.00826211,  0.00681123,  0.00359201, -0.00190429, -0.00361719,
       -0.00606572, -0.00093576,  0.00284142, -0.00451833,  0.00577845,
        0.00671156, -0.00066289,  0.00478757, -0.0056951 , -0.00901141,
       -0.0005129 ,  0.00673506,  0.00058463, -0.00919403,  0.00350139,
        0.00254778,  0.00077734,  0.0096749 , -0.00078415, -0.00972068,
        0.00720197, -0.00462728, -0.00856202, -0.00897036,  0.00436376],
      dtype=float32)